## Design a recursion function 

### Interpret Recursion

Consider we want to solve a problem $F(X)$, we can decompose it into several small problems: $x_0 \in X, x_1 \in X, ....$

Base case: **The solution of the minimum scale problem is the base case of the recursive function, through which the <u>pop operation</u> of the stack is realized until the stack is empty, and the problem is solved.**

Recursion relationship: **Writing out the recursive relationship is equivalent to implementing the push operation of the stack. For languages ​​that support recursion, the underlying data structure is a stack, so the implicit stack is used to solve the final result.**

**Recursion is a programming technique, not an algorithm**. *Divide and conquer, dynamic programming, and backtracking* are algorithms. The relationship between the two is that the most commonly used implementation of these three algorithms is recursion.<br/>
In both the divide and conquer algorithm and the dynamic programming algorithm, it is necessary to disassemble the large problem into sub-problems.

### **wrapper funtion**


A wrapper function is a function that is directly called but does not recurse itself, instead calling a separate *auxiliary function* which actually does the recursion.

Wrapper functions can be used to <u>validate parameters (so the recursive function can skip these)</u>, perform initialization (allocate memory, initialize variables), particularly for auxiliary variables such as "level of recursion" or <u>partial computations for memoization</u>, and handle exceptions and errors.

> Wrapper functions can reduce the number of parameters in the recursive call, which reduces the stack usage, as all parameters are pushed on the stack for each call, and if there are many parameters and many layers of recursion, there could be a stack overflow. Any parameters that do no change for the recursive call can be left in an enclosing lexical scope or passed altogether as a reference to a structure.

```c
\\ Ordinary
int fac1(int n) {
   if (n <= 0)
      return 1;
   else
      return fac1(n-1)*n;
}
```

```c
\\ Short-circuit recursion 
static int fac2(int n) {
   // assert(n >= 2);
   if (n == 2)
      return 2;
   else
      return fac2(n-1)*n;
}
int fac2wrapper(int n) {
   if (n <= 1)
      return 1;
   else
      return fac2(n);
}
```

**Short-circuiting the base case**:
Short-circuiting the base case, also known as **arm's-length recursion**, consists of checking the base case before making a recursive call – *i.e.*, checking if the next call will be the base case, instead of calling and then checking for the base case.

For example, in the factorial function, <u>properly the base case is 0! = 1, while immediately returning 1 for 1! is a short circuit</u>, and may miss 0; this can be mitigated by a wrapper function. 

- Short-circuiting is particularly done for efficiency reasons, to avoid the overhead of a function call that immediately returns.
- Short-circuiting is primarily a concern when many base cases are encountered, such as `Null` pointers in a tree, which can be linear in the number of function calls, hence significant savings for $O(n)$ algorithms; this is illustrated below for a depth-first search. 
- Short-circuiting on a tree corresponds to considering <u>a leaf (non-empty node with no children) as the base case</u>, rather than considering an empty node as the base case. If there is only a single base case, such as in computing the factorial, short-circuiting provides only O(1) savings. 


### DFS

The standard recursive algorithm for a DFS is:

- base case: If current node is Null, return false
- recursive step: otherwise, check value of current node, return true if match, otherwise recurse on children

In short-circuiting, this is instead:

- check value of current node, return true if match,
- otherwise, on children, if not Null, then recurse.


In the case of a perfect binary tree of height $h$, there are $2^{h+1}−1$ nodes and $2^{h+1}$ `Null` pointers as children ($2$ for each of the $2^h$ leaves), so short-circuiting cuts the number of function calls in half in the worst case. 

```c
bool tree_contains(struct node *tree_node, int i) {
    if (tree_node == NULL)
        return false;  // base case
    else if (tree_node->data == i)
        return true;
    else
        return tree_contains(tree_node->left, i) ||
               tree_contains(tree_node->right, i);
}

```

```c
// Wrapper function to handle empty tree
bool tree_contains(struct node *tree_node, int i) {
    if (tree_node == NULL)
        return false;  // empty tree
    else
        return tree_contains_do(tree_node, i);  // call auxiliary function
}

// Assumes tree_node != NULL
bool tree_contains_do(struct node *tree_node, int i) {
    if (tree_node->data == i)
        return true;  // found
    else  // recurse
        return (tree_node->left  && tree_contains_do(tree_node->left,  i)) ||
               (tree_node->right && tree_contains_do(tree_node->right, i));
}
```

## Learn with Practice

---
### Minimum depth of tree

Example:
1. - Input:  root = `[3,9,20,null,null,15,7]` 
   - output:  `2`
2. - Input: root = `[2,null,3,null,4,null,5,null,6]`
   - output: `5`

- Base case: if `null`, return `0`; if leaf nodes, return `1`;
- Recursion Relationship: 
  1. if both left child and right child, depth = smaller depth + 1;
  2. if only left/right child, depth = depth of left/right child + 1;
  3. if neither left child nor right child, depth = 1;

In [10]:
from binarytree import build, build2

In [83]:
arr =  [3, 9, 20, None, None, 15, 7]
arr2 = [2, None, 3, None, 4, None, 5, None, 6]
root = build(arr)
root2 = build2(arr2)

#### DFS Solution

In [84]:
def min_depth(root):
    # empty tree/subtree
    if root is None: return 0  
    ld = min_depth(root.left)
    rd = min_depth(root.right)
    # has only left/right child, or probably is a leaf node
    if root.left is None or root.right is None:
        return ld + rd + 1  # ld or rd must be 0 (both 0 if a leaf node)
    # both left and right child
    return min(ld, rd) + 1 

print(min_depth(root))
print(min_depth(root2))

2
5


In [93]:
def minDepth(root) -> int:
    if not root: return 0
    elif not root.left: return minDepth(root.right) + 1
    elif not root.right: return minDepth(root.left) + 1
    else: return min(minDepth(root.left), minDepth(root.right)) + 1

print(minDepth(root))
print(minDepth(root2))

2
5


- **Time complexity**: $O(N)$
- **Space complexity**: $O(H)$, where $H$ is the height of the tree.
  - Worst case: the tree is chain-like, $H = N$, *i.e.*, $O(N)$;
  - Average case: $H = \log N$, *i.e.*, $O(\log N)$.

#### BFS Solution

In [89]:
def min_depth_bfs(root):
    if root is None: return 0
    queue = [(root, 1)]
    while queue:
        node, depth = queue.pop(0)
        if not node.left and not node.right:
            return depth
        if node.left:
            queue.append((node.left, depth + 1))
        if node.right:
            queue.append((node.right, depth + 1))
            
min_depth_bfs(root), min_depth_bfs(root2)

(2, 5)

In [92]:
def min_depth_bfs(root):
    if not root: return 0
    queue = [root]for _ in range(num):
            node = queue.pop(0)
    depth = 1
    while queue:
        num = len(queue)
        for _ in range(num):
            node = queue.pop(0)
            if not node.left and not node.right:
                return depth
            if node.left:
                queue.append(node.left)
            if node.right:
                queue.append(node.right) 
        depth += 1
        
min_depth_bfs(root), min_depth_bfs(root2)

(2, 5)

- **Time complexity**: $O(N)$
- **Space complexity**: $O(N)$, the space complexity depends on the size of the queue.

---
### Path Sum

**Requirements**:

gives you the root node of the binary tree root and an integer representing the target sum `targetSum`. Determine whether there is a path from the root node to the leaf node in the tree , and the sum of all node values ​​on this path is equal to the target sum `targetSum`. If exists, return true; otherwise, return `false`.

![](https://assets.leetcode.com/uploads/2021/01/18/pathsum1.jpg)
```
  input:  root = [5,4,8,11,null,13,4,7,2,null,null,null,1], targetSum = 22
  output:  true 
  Explanation:  The root node to leaf node path equal to the target sum is shown in the figure above. 
 
```


**Analysis**:
- Basic case:
- Recursion relationship:

In [103]:
arr = [5,4,8,11,None,13,4,7,2,None,None,None,1]
root = build(arr)
targetSum = 22 
print(root)


         5_____
        /      \
    ___4     ___8
   /        /    \
  11       13     4
 /  \        \
7    2        1



#### DFS Solution

In [98]:
def pathSum(root, targetSum):
    if not root: return False
    if not root.left and not root.right: 
        return (root.value == targetSum)
    return pathSum(root.left, targetSum-root.value) \
        or pathSum(root.right, targetSum-root.value) 
        
pathSum(root, targetSum)

True

#### BFS Solution

In [109]:
def pathSum(root, targetSum):
    if not root: return False
    queue = [(root, root.value)]
    while queue:
        node, value = queue.pop(0)
        if not node.left and not node.right:
            if targetSum == value:
                return True
            continue
        if node.left:
            queue.append((node.left, value + node.left.value))
        if node.right:
            queue.append((node.right, value + node.right.value))

pathSum(root, targetSum)

True

---
### Symmetric binary tree

**Requirements**:

```
Input:  root = [1,2,2,3,4,4,3] 
  output:  true 
 ```

```
Input:  root = [1,2,2,null,3,null,3] 
  output:  false 
 ```

**Analysis**:
- Basic case:
- Recursion relationship:

In [113]:
arr1 = [1,2,2,3,4,4,3]
root1 = build(arr1)
arr2 = [1,2,2,None,3,None,3]
root2 = build(arr2)

#### Recursion / DFS

In [115]:
def is_symmetric(root):
    if root is None: return False
    def symmetric_helper(left, right):
        if left is None and right is None:
            return True
        elif left is None or right is None:
            return False
        return left.value == right.value and \
            symmetric_helper(left.right, right.left) and \
            symmetric_helper(left.left, right.right)
    return symmetric_helper(root.left, root.right)

is_symmetric(root1), is_symmetric(root2)

(True, False)

#### Iterative / Queue

In [ ]:
def is_symmetric(root):
    queue = [root]
    while queue:
        size = len(queue)
        tmp_queue = [0] * size**2
        for i in range(size // 2):
            node_left = queue.pop(0)
            node_right = queue.pop()
            if node_left is None and node_right is None:
                tmp_queue[i*2:i*2+1] = None
                tmp_queue[-(i*2+2):-(i*2+1)] = None
                continue
            elif not node_left or not node_right:
                return False
            elif node_left.value != node_right.value:
                return False
            else:
                tmp_queue[i*2], tmp_queue[i*2+1] = node_left.left, node_left.right
                tmp_queue[-(i*2+2)], tmp_queue[-(i*2+1)] = node_right.left, node_right.left 
                
        queue = tmp_queue
    return False


---
### Full permutation

**Requirements**:


**Analysis**:
- Basic case:
- Recursion relationship: